# MC and portfolio optimization

One of the assumptions of Modern Portfolio Theory (MPT) is that the mean and covariance matrix of returns are known. Clearly that is not the case and we have to use some estimates. The accuracy of the estimates used has a big effect on the resulting portfolio.

(The $paper^{[2]}$)Section II of the paper demonstrates several empirical regularities found in financial data:

* Volatility is time-varying.
* Correlations are time-varying.
* Financial data has fat-tails.

In the paper authors propose using weighted estimators, which give more weight to newer datapoints and less weight to older datapoints. Formula for calculating weighted covariance is provided below. Formula in the paper is a little different — it doesn’t contain expectations of returns because they are assumed to be zero.

$$ 
\sigma_{ij} = \frac{\sum^T_{s=0}(w_{T-s}(r_{i,T-s} - \mathbb{E[r_i]})(r_{j,T-s} - \mathbb{E[r_j]}))}{\sum^T_{s=0}W_{T-s}} 
$$
* $r_{i, t}$: return of asset $i$ at time $t$
* $w_t$: weight of a data point at time $t$

Weighted average of returns
$$
\mu_i = \frac{\sum^T_{s=0}{(W_{T-s} r_{i, T-s})}}{\sum^T_{s=0}{W_{T-s}}}
$$

$w_t$ should be a decaying function of time:
$$ w_t = (1-d)w_{t-1}$$
$d:$ decay rate

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from tqdm import tqdm
from scipy.optimize import minimize

%config InlineBackend.figure_format = 'retina'
sns.set_style('darkgrid')

In [2]:
def calculate_metrics(cumret):
    '''
    calculate performance metrics from cumulative returns
    '''
    total_return = (cumret[-1] - cumret[0])/cumret[0]
    apr = (1+total_return)**(52/len(cumret)) - 1
    rets = pd.DataFrame(cumret).pct_change()
    rets.iloc[0] = 0
    sharpe = np.sqrt(52) * np.nanmean(rets) / np.nanstd(rets)
    
    # maxdd and maxddd
    highwatermark=np.zeros(cumret.shape)
    drawdown=np.zeros(cumret.shape)
    drawdownduration=np.zeros(cumret.shape)
    for t in np.arange(1, cumret.shape[0]):
        highwatermark[t]=np.maximum(highwatermark[t-1], cumret[t])
        drawdown[t]=cumret[t]/highwatermark[t]-1
        if drawdown[t]==0:
            drawdownduration[t]=0
        else:
            drawdownduration[t]=drawdownduration[t-1]+1
    maxDD=np.min(drawdown)
    maxDDD=np.max(drawdownduration)
    
    return total_return, apr, sharpe, maxDD, maxDDD

In [ ]:
spy = yf.download('SPY', start='2018-01-01', end='2022-12-31')['Adj Close']
spy_ret = spy.pct_change().dropna()
spy_hist_std = spy_ret.std()*np.sqrt(252) # annualized historical sd
spy_rolling_std = spy_ret.rolling(100).std().dropna()*np.sqrt(252) # annualized rolling sd (100 days)

plt.figure(figsize=(18,6))
plt.plot(spy_rolling_std, label='100-day rolling')
plt.axhline(spy_hist_std, label='historical', color='m', linestyle='dashed')
plt.legend()

## Resources

* https://medium.com/@financialnoob/portfolio-optimization-with-weighted-mean-and-covariance-estimators-de6dda2cab3d
* http://www.andreisimonov.com/4106/pdf/GS_Estimating_covariance_matrices.pdf